# SDSU Data

In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
df = pd.read_excel(io='SDSU_mech_eng_classes.xlsx', header=1)

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Department      96 non-null     object 
 1   Subject         96 non-null     object 
 2   Course #        96 non-null     int64  
 3   Suffix          13 non-null     object 
 4   Catalog #       0 non-null      float64
 5   Title           96 non-null     object 
 6   Section         96 non-null     int64  
 7   Schedule #      96 non-null     object 
 8   Units           96 non-null     int64  
 9   Session         96 non-null     object 
 10  Seats           96 non-null     int64  
 11  Available       96 non-null     int64  
 12  Total Enrolled  96 non-null     int64  
 13  Blocked         0 non-null      float64
 14  Footnotes       89 non-null     object 
 15  SDSU Enrolled   0 non-null      float64
 16  EE Enrolled     0 non-null      float64
 17  SDSU Audit      0 non-null      float

In [4]:
df = df.drop(['Catalog #', 'Blocked', 'SDSU Enrolled', 'EE Enrolled', 'SDSU Audit', 'EE Audit'], axis=1)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Department      96 non-null     object 
 1   Subject         96 non-null     object 
 2   Course #        96 non-null     int64  
 3   Suffix          13 non-null     object 
 4   Title           96 non-null     object 
 5   Section         96 non-null     int64  
 6   Schedule #      96 non-null     object 
 7   Units           96 non-null     int64  
 8   Session         96 non-null     object 
 9   Seats           96 non-null     int64  
 10  Available       96 non-null     int64  
 11  Total Enrolled  96 non-null     int64  
 12  Footnotes       89 non-null     object 
 13  Meeting Type    96 non-null     object 
 14  Building        84 non-null     object 
 15  Room            84 non-null     float64
 16  Start Time      84 non-null     float64
 17  End Time        84 non-null     float

Convert to special Int64 type because there are null values in the columns. (pandas nullable integrer data type)

In [5]:
df['Room'] = df['Room'].astype('Int64')
df['Start Time'] = df['Start Time'].astype('Int64')
df['End Time'] = df['End Time'].astype('Int64')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Department      96 non-null     object
 1   Subject         96 non-null     object
 2   Course #        96 non-null     int64 
 3   Suffix          13 non-null     object
 4   Title           96 non-null     object
 5   Section         96 non-null     int64 
 6   Schedule #      96 non-null     object
 7   Units           96 non-null     int64 
 8   Session         96 non-null     object
 9   Seats           96 non-null     int64 
 10  Available       96 non-null     int64 
 11  Total Enrolled  96 non-null     int64 
 12  Footnotes       89 non-null     object
 13  Meeting Type    96 non-null     object
 14  Building        84 non-null     object
 15  Room            84 non-null     Int64 
 16  Start Time      84 non-null     Int64 
 17  End Time        84 non-null     Int64 
 18  Days        

In [6]:
# df['Course #'] = df['Course #'].astype('int32')

In [7]:
# print(df.iloc[0])
# print(df.iloc[0]['Subject'])
# print(df.iloc[0][1])
print(tuple(df.iloc[0].to_list()))

('Mechanical Engineering', 'M E', 190, nan, 'COMPUTER-AIDED DESIGN', 1, 22216, 2, 'M1', 40, 18, 22, 'IA', 'Lecture', 'E', 221, 800, 850, 'M', 'C. SAGHAFI')


In [8]:
sqlite3.register_adapter(np.int64, lambda val: int(val))
# sqlite3.register_adapter(np.int32, lambda val: int(val))

In [9]:
con = sqlite3.connect("SDSU_mech_eng_classes.sqlite")

In [10]:
cur = con.cursor()

In [11]:
cur.execute('DROP TABLE IF EXISTS Classes')
cur.execute('''CREATE TABLE Classes 
            (Department TEXT, Subject TEXT, CourseNum INTEGER, Suffix TEXT, Title TEXT, Section INTEGER, ScheduleNum TEXT, 
            Units INTEGER, Session TEXT, Seats INTEGER, Available INTEGER, TotalEnrolled INTEGER, Footnotes TEXT, 
            MeetingType TEXT, Building TEXT, Room INTEGER, StartTime INTEGER, EndTime INTEGER, Days TEXT, Instructor TEXT)''')


In [15]:
for i in range(len(df)):

    try:
        cur.execute('''INSERT INTO Classes
                    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', tuple(df.iloc[i].to_list()))
#         print('Done', i)
    except:
#         print('Failed on', i)

con.commit()

IndentationError: expected an indented block (<ipython-input-15-49a9af0def6d>, line 10)

In [13]:
con.close()

In [16]:
df.tail(15)

,Department,Subject,Course #,Suffix,Title,Section,Schedule #,Units,Session,Seats,Available,Total Enrolled,Footnotes,Meeting Type,Building,Room,Start Time,End Time,Days,Instructor
81,Mechanical Engineering,M E,651,NaN,ADVANCED THERMODYNAMICS,1,22278,3,M1,40,30,10,"ZL,ZM",Lecture,HH,222,1700,1815,TTH,K. WOOD
82,Mechanical Engineering,M E,657,NaN,CONVECTION HEAT TRANSFER,1,22280,3,M1,45,39,6,"NC,ZL,ZM",Lecture,E,201,1900,2015,MW,F. MILLER
83,Mechanical Engineering,M E,681,NaN,BIOMATERIALS,1,22282,3,M1,45,33,12,"ZL,ZM",Lecture,LH,408,1730,1845,TTH,Y. YANG
84,Mechanical Engineering,M E,696,NaN,COMP TRANSPRT PHENOMENA,3,22284,3,M1,30,26,4,"ZL,ZM",Lecture,P,147,1700,1815,MW,A. BHALLA
85,Mechanical Engineering,M E,696,NaN,COMP TRANSPRT PHENOMENA,4,39108,3,M1,40,32,8,"NC,ZM",Lecture,LL,406,1400,1515,TTH,W. XU
86,Mechanical Engineering,M E,696,NaN,COMP TRANSPRT PHENOMENA,5,39383,3,M1,42,37,5,"ZL,ZM",Lecture,HH,206,1700,1815,MW,G. YOUSSEF
87,Mechanical Engineering,M E,797,NaN,RESEARCH,1,*****,1,M1,0,0,0,"F,J,X,ZL,ZM",Supervised,NaN,<NA>,<NA>,<NA>,ARR,F. MILLER
88,Mechanical Engineering,M E,797,NaN,RESEARCH,2,*****,2,M1,0,0,0,"F,J,X,ZL,ZM",Supervised,NaN,<NA>,<NA>,<NA>,ARR,F. MILLER
89,Mechanical Engineering,M E,797,NaN,RESEARCH,3,*****,3,M1,0,0,0,"F,J,X,ZL,ZM",Supervised,NaN,<NA>,<NA>,<NA>,ARR,F. MILLER
90,Mechanical Engineering,M E,798,NaN,SPECIAL STUDY,1,*****,1,M1,0,0,0,"F,J,X,ZL,ZM",Supervised,NaN,<NA>,<NA>,<NA>,ARR,F. MILLER


In [14]:
# cur.execute('SELECT * FROM Classes WHERE Title = \'COMPUTER-AIDED DESIGN\'')
# cur.fetchall()